<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ..
%mkdir spark

/


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget -P /spark/ https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-23 13:57:02--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.233.100, 64.50.236.52, 140.211.166.134, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.233.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  6.26MB/s    in 7m 39s  

2021-07-23 14:04:42 (479 KB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [4]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz 
%mv spark-3.0.3-bin-hadoop3.2 /spark/




In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark/spark-3.0.3-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
%cd /content/
%mkdir /chapterData 
%cd /chapterData
%mkdir retail-data
%cd retail-data/
%mkdir by-day
%cd by-day

/content
/chapterData
/chapterData/retail-data
/chapterData/retail-data/by-day


In [9]:
%pwd


'/chapterData/retail-data/by-day'

In [10]:
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-05.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-06.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-07.csv'


--2021-07-23 14:04:56--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-05.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246056 (240K) [text/plain]
Saving to: ‘2010-12-05.csv’

010-12-05.csv       100%[===================>] 240.29K  --.-KB/s    in 0.03s   

2021-07-23 14:04:56 (7.19 MB/s) - ‘2010-12-05.csv’ saved [246056/246056]

--2021-07-23 14:04:56--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-06.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected

In [11]:

!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-03.csv'




--2021-07-23 14:04:57--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275001 (269K) [text/plain]
Saving to: ‘2010-12-01.csv’

010-12-01.csv       100%[===================>] 268.56K  --.-KB/s    in 0.04s   

2021-07-23 14:04:58 (6.97 MB/s) - ‘2010-12-01.csv’ saved [275001/275001]

--2021-07-23 14:04:58--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected

In [12]:
%mv /chapterData /content/

**Prepping data**

In [52]:
staticDF = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.load("/content/chapterData/retail-data/by-day/*.csv")

In [53]:
staticSchema = staticDF.schema

In [54]:
staticDF.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdom|
|   537226|    20802|SMALL GLASS SUNDA...|       6|2010-12-06 08:34:00|     1.65|   15987.0|United Kingdom|
|   537226|    22052|VINTAGE CARAVAN G...|      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    22705|   WRAP GREEN PEARS |      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    20781|GOLD EA

**Static DF example to learn more about streaming**

In [55]:
dayOfLargePurchase = staticDF.groupby("InvoiceDate").sum("UnitPrice").withColumnRenamed("sum(UnitPrice)", "SumOFUnitPrice")
dayOfLargePurchase.sort(dayOfLargePurchase.InvoiceDate.desc()).take(10)

[Row(InvoiceDate='2010-12-07 18:36:00', SumOFUnitPrice=3118.1100000000033),
 Row(InvoiceDate='2010-12-07 17:19:00', SumOFUnitPrice=39.16),
 Row(InvoiceDate='2010-12-07 17:05:00', SumOFUnitPrice=1.95),
 Row(InvoiceDate='2010-12-07 16:43:00', SumOFUnitPrice=128.26999999999998),
 Row(InvoiceDate='2010-12-07 16:42:00', SumOFUnitPrice=35.62),
 Row(InvoiceDate='2010-12-07 16:21:00', SumOFUnitPrice=10.95),
 Row(InvoiceDate='2010-12-07 16:20:00', SumOFUnitPrice=24.599999999999998),
 Row(InvoiceDate='2010-12-07 15:59:00', SumOFUnitPrice=63.300000000000004),
 Row(InvoiceDate='2010-12-07 15:54:00', SumOFUnitPrice=153.59),
 Row(InvoiceDate='2010-12-07 15:51:00', SumOFUnitPrice=6706.71)]

In [56]:
from pyspark.sql.functions import window , column , desc , col 
staticDataFrame = staticDF.selectExpr("CustomerID", "(UnitPrice*Quantity) as total_purchase_value", "InvoiceDate").groupBy(\
    col("CustomerId"),\
    window(col("InvoiceDate"),"1 day"))\
.sum("total_purchase_value")\
.withColumnRenamed("sum(total_purchase_value)", "total_purchase_value")\
.show(10)



+----------+--------------------+--------------------+
|CustomerId|              window|total_purchase_value|
+----------+--------------------+--------------------+
|   13408.0|[2010-12-01 00:00...|  1024.6800000000003|
|   17460.0|[2010-12-01 00:00...|                19.9|
|   16950.0|[2010-12-07 00:00...|               172.0|
|   13269.0|[2010-12-05 00:00...|              351.43|
|   12647.0|[2010-12-05 00:00...|               372.0|
|   15235.0|[2010-12-01 00:00...|                79.5|
|   13495.0|[2010-12-06 00:00...|  510.94999999999993|
|   14506.0|[2010-12-05 00:00...|  269.69999999999993|
|   14911.0|[2010-12-06 00:00...|  1182.5000000000002|
|   13295.0|[2010-12-02 00:00...|               -3.25|
+----------+--------------------+--------------------+
only showing top 10 rows



**Learning Streaming**

In [57]:
readData = spark.readStream.schema(staticSchema)\
.option("maxFilesTrigger",1)\
.option("header", "true")\
.format("csv")\
.load("/content/chapterData/retail-data/by-day/*.csv")

In [19]:
readData.isStreaming

True

In [20]:
purchaseCustomerPerHour = readData.selectExpr("CustomerId", "(UnitPrice*Quantity) as Total_Cost", "InvoiceDate")\
.groupBy(col("CustomerId")\
,window(col("InvoiceDate"), "1 day"))\
.sum("Total_Cost")

In [21]:
purchaseCustomerPerHr= purchaseCustomerPerHour.writeStream\
.format("memory")\
.queryName("Customer_Purchase")\
.outputMode("complete")\
.start()

In [35]:
spark.sql("select * from Customer_Purchase").show(10)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(Total_Cost)|
+----------+--------------------+------------------+
|   13269.0|[2010-12-05 00:00...|            351.43|
|   12647.0|[2010-12-05 00:00...|             372.0|
|   17460.0|[2010-12-01 00:00...|              19.9|
|   16950.0|[2010-12-07 00:00...|             172.0|
|   13408.0|[2010-12-01 00:00...|1024.6800000000003|
|   15235.0|[2010-12-01 00:00...|              79.5|
|   14506.0|[2010-12-05 00:00...|269.69999999999993|
|   13495.0|[2010-12-06 00:00...|510.94999999999993|
|   14911.0|[2010-12-06 00:00...|1182.5000000000002|
|   12841.0|[2010-12-03 00:00...|            294.25|
+----------+--------------------+------------------+
only showing top 10 rows



**Learning about Machine Learning!**

In [58]:
staticDF.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [59]:
from pyspark.sql.functions import date_format, col
preppedDateData  = staticDF.na.fill(0).withColumn("day_of_week", \
                               date_format(col("InvoiceDate"), "EEEE"))\
                               .coalesce(5)

In [25]:
#staticDF.filter("InvoiceDate is null").show(100)

**Checking the various dates to create train and test sets**

In [60]:
preppedDateData.select("InvoiceDate")\
.distinct()\
.show(10)

+-------------------+
|        InvoiceDate|
+-------------------+
|2010-12-07 13:29:00|
|2010-12-02 11:45:00|
|2010-12-07 13:30:00|
|2010-12-02 18:49:00|
|2010-12-01 17:29:00|
|2010-12-02 18:45:00|
|2010-12-03 12:28:00|
|2010-12-02 15:26:00|
|2010-12-01 09:34:00|
|2010-12-07 13:25:00|
+-------------------+
only showing top 10 rows



In [62]:
trainingData = preppedDateData.where("InvoiceDate < '2010-12-03'")
testData = preppedDateData.where("InvoiceDate >= '2010-12-03'")

In [68]:
trainingData.count()
trainingData.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|  Wednesday|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Wednesday|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|  Wednesday|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Wednesday|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|  Wednesday|
|   536365|    22752|SET 7 BABUSHKA NE..

In [64]:
testData.count()

11768

In [65]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="day_of_week", outputCol="day_of_week_index")

In [45]:
from pyspark.ml.feature import OneHotEncoder 
encoder = OneHotEncoder(inputCol="day_of_week_index", outputCol="day_of_week_encoded")

In [75]:
#doesnt work!
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(outputCol="features")
vectorAssembler.setInputCols(["UnitPrice,Quantity,day_of_week_encoded"])
vectorAssembler.getOutputCol(col("UnitPrice"))

TypeError: ignored

In [47]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline().setStages([indexer,encoder,vectorAssembler])

In [66]:
fittedPipeline = transformationPipeline.fit(trainingData)

In [67]:
from pyspark.ml import Transformer
transformedPipeline = fittedPipeline.transform(trainingData)

IllegalArgumentException: ignored